# Raw → Processed Cleaning
This notebook cleans the **UNSD Methodology** and **ROR** CSVs from `raw_data/` and writes cleaned outputs to `processed_data/`.

**What it does**
1. UNSD: keep `ISO-alpha2 Code`, `Country or Area`, and `Subregion` as `coalesce(Intermediate Region, Subregion)`; trim; uppercase ISO; drop duplicates/empties.
2. ROR: keep `name→institution_name`, `aliases`, `acronyms`, `country code`, `country name`; trim; parse list-like arrays; uppercase codes; drop duplicates/empties.

You can re-run this as new raw files arrive—it's resilient to minor column-name variants.

In [1]:

import os, re, glob, json
from typing import List, Optional
import pandas as pd
import numpy as np

RAW_DIR = r"raw_data"
PROCESSED_DIR = r"processed_data"
os.makedirs(PROCESSED_DIR, exist_ok=True)

def normalize_col(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[\s/_\-]+", " ", s)
    s = re.sub(r"[^0-9a-z ]+", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def find_first_column(df: pd.DataFrame, must_contain: List[str], optional_contains: List[str]=[]) -> Optional[str]:
    norm_map = {col: normalize_col(col) for col in df.columns}
    for col, norm in norm_map.items():
        if all(term in norm for term in must_contain):
            if optional_contains:
                if any(opt in norm for opt in optional_contains):
                    return col
            else:
                return col
    return None

def coalesce_values(*series):
    df = pd.concat(series, axis=1)
    def pick(row):
        for val in row:
            if pd.notna(val) and str(val).strip() != "":
                return str(val).strip()
        return np.nan
    return df.apply(pick, axis=1)

def parse_listlike_to_joined(s: pd.Series) -> pd.Series:
    out = []
    for val in s.astype(str).tolist():
        v = val.strip()
        if v.startswith("[") and v.endswith("]"):
            try:
                arr = json.loads(v)
                if isinstance(arr, list):
                    items = [str(x).strip() for x in arr if str(x).strip() != ""]
                    out.append(" | ".join(items) if items else np.nan)
                    continue
            except Exception:
                pass
        out.append(v if v != "" else np.nan)
    return pd.Series(out, index=s.index, dtype="object")


In [3]:

# --- Clean UNSD Methodology ---
# Try to locate an UNSD CSV in RAW_DIR
unsd_path = r"raw_data\UNSD — Methodology.csv"
print("UNSD path:", unsd_path)

if unsd_path:
    dfu = pd.read_csv(unsd_path, encoding="utf-8-sig")
    for col in dfu.columns:
        if dfu[col].dtype == object:
            dfu[col] = dfu[col].astype(str).str.strip().replace({"nan": np.nan})

    col_iso2 = (find_first_column(dfu, ["alpha","2","code"], ["iso","alpha2","alpha 2"]) or
                find_first_column(dfu, ["iso","alpha","2"]) or
                ("ISO-alpha2 Code" if "ISO-alpha2 Code" in dfu.columns else None))

    col_country = (find_first_column(dfu, ["country","area"]) or
                   find_first_column(dfu, ["country"]) or
                   ("Country or Area" if "Country or Area" in dfu.columns else None))

    col_intermediate = (find_first_column(dfu, ["intermediate","region"]) or
                        find_first_column(dfu, ["intermediate"]))
    col_subregion = (find_first_column(dfu, ["subregion"]) or
                     find_first_column(dfu, ["sub","region"]))

    work = pd.DataFrame()
    work["iso_alpha2"] = dfu[col_iso2].astype(str).str.strip().str.upper().replace({"": np.nan}) if col_iso2 else np.nan
    work["country_or_area"] = dfu[col_country].astype(str).str.strip().replace({"": np.nan}) if col_country else np.nan

    if (col_intermediate is not None) or (col_subregion is not None):
        inter = dfu[col_intermediate].astype(str).str.strip().replace({"": np.nan}) if col_intermediate else pd.Series([np.nan]*len(dfu))
        subr = dfu[col_subregion].astype(str).str.strip().replace({"": np.nan}) if col_subregion else pd.Series([np.nan]*len(dfu))
        work["subregion"] = coalesce_values(inter, subr)
    else:
        work["subregion"] = np.nan

    work = work.dropna(subset=["iso_alpha2"])
    work = work[work["iso_alpha2"].str.len() == 2]
    work["country_or_area"] = work["country_or_area"].apply(lambda x: x if pd.isna(x) else x.strip())
    work["subregion"] = work["subregion"].apply(lambda x: x if pd.isna(x) else x.strip())
    work = work.drop_duplicates().sort_values(["subregion","country_or_area","iso_alpha2"], na_position="last").reset_index(drop=True)

    out_path = os.path.join(PROCESSED_DIR, "unsd_methodology_clean.csv")
    work.to_csv(out_path, index=False, encoding="utf-8")
    print("Wrote:", out_path, "rows:", len(work))
else:
    print("No UNSD file found in", RAW_DIR)


UNSD path: raw_data\UNSD — Methodology.csv
Wrote: processed_data\unsd_methodology_clean.csv rows: 247


In [5]:

ror_path = r"raw_data\v1.73-2025-10-28-ror-data.csv"
print("ROR path:", ror_path)

if ror_path:
    dfr = pd.read_csv(ror_path, encoding="utf-8-sig")

    for col in dfr.columns:
        if dfr[col].dtype == object:
            dfr[col] = dfr[col].astype(str).str.strip().replace({"nan": np.nan})

    # identify columns
    col_name = "name" if "name" in dfr.columns else None
    if not col_name:
        col_name = [c for c in dfr.columns if "name" in c.lower() and "country" not in c.lower()]
        col_name = col_name[0] if col_name else None

    col_aliases = "aliases" if "aliases" in dfr.columns else None
    if not col_aliases:
        col_aliases = [c for c in dfr.columns if "alias" in c.lower()]
        col_aliases = col_aliases[0] if col_aliases else None

    col_acronyms = "acronyms" if "acronyms" in dfr.columns else None
    if not col_acronyms:
        col_acronyms = [c for c in dfr.columns if "acronym" in c.lower()]
        col_acronyms = col_acronyms[0] if col_acronyms else None

    # NEW: types
    col_types = "types" if "types" in dfr.columns else None
    if not col_types:
        type_candidates = [c for c in dfr.columns if "type" in c.lower() and "country" not in c.lower()]
        col_types = type_candidates[0] if type_candidates else None

    # country code/name
    cc_candidates = ["country.code","country_code","countrycode","country code"]
    cn_candidates = ["country.name","country_name","countryname","country name"]
    col_ccode = next((c for c in cc_candidates if c in dfr.columns), None)
    col_cname = next((c for c in cn_candidates if c in dfr.columns), None)
    if not col_ccode:
        guess = [c for c in dfr.columns if "country" in c.lower() and "code" in c.lower()]
        col_ccode = guess[0] if guess else None
    if not col_cname:
        guess = [c for c in dfr.columns if "country" in c.lower() and "name" in c.lower()]
        col_cname = guess[0] if guess else None

    # Build output
    r = pd.DataFrame()
    r["institution_name"] = dfr[col_name].astype(str).str.strip().replace({"": np.nan}) if col_name else np.nan
    r["aliases"] = parse_listlike_to_joined(dfr[col_aliases]) if col_aliases else np.nan
    r["acronyms"] = parse_listlike_to_joined(dfr[col_acronyms]) if col_acronyms else np.nan
    r["types"] = parse_listlike_to_joined(dfr[col_types]) if col_types else np.nan  # <-- added
    r["country_code"] = dfr[col_ccode].astype(str).str.upper().str.strip().replace({"": np.nan}) if col_ccode else np.nan
    r["country_name"] = dfr[col_cname].astype(str).str.strip().replace({"": np.nan}) if col_cname else np.nan

    # Clean up
    r = r.dropna(subset=["institution_name"])
    r["institution_name"] = r["institution_name"].str.replace(r"\s+", " ", regex=True).str.strip()
    r["country_code"] = r["country_code"].apply(lambda x: x if (pd.isna(x) or len(str(x)) <= 3) else str(x)[:3].strip())
    r["country_name"] = r["country_name"].str.replace(r"\s+", " ", regex=True).str.strip()
    r = r.drop_duplicates().sort_values(["country_name","institution_name"], na_position="last").reset_index(drop=True)

    out_path = os.path.join(PROCESSED_DIR, "ror_data_clean.csv")
    r.to_csv(out_path, index=False, encoding="utf-8")
    print("Wrote:", out_path, "rows:", len(r))
else:
    print("No ROR file found in", RAW_DIR)


ROR path: raw_data\v1.73-2025-10-28-ror-data.csv
Wrote: processed_data\ror_data_clean.csv rows: 119690
